In [5]:
import openai
import pandas as pd

OPEN_API_KEY = ''

openai.api_key = OPEN_API_KEY

def get_completion(paragraph, level):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "[역할] 당신은 초등학교 국어 선생님입니다. 해당 지문을 확인한 후 빈칸을 뚫어 추론 유형의 4지선다 문제를 출제하세요."},
                {"role": "user", "content": f"[방법]"
                                            f" 아래 주어진 지문 내용을 바탕으로 {level} 수준의 4지선다 형태의 추론 유형의 객관식 문제 1개를 출제하세요. 반드시 아래 Output의 유형에 맞는 구성으로 빈칸을 생성하고 해당 내용을 정답으로 만드세요. 요구한 결과물만 출력해줘"
                                            f"[Input] "
                                            f"[지문]  "
                                            f"{paragraph} "
                                            f"[난이도] - {level}"
                                            f""
                                            f"[Output]"
                                            f"{example}"}
            ],
            model="gpt-4o-mini"
        )
        # print(response)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

news = pd.read_csv("../data/processed/news/result/news_level.csv", sep=',')

example = """
{
    "question": "빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요",
    "text": "활활 타오르는 태양이 내뿜는 빛은 지구로 쏟아져요. 이런 태양 빛을 모아 전기를 만드는 방법을 ‘태양광 발전’이라고 합니다. 태양광 발전은 지구온난화를 심화시키는 탄소 배출을 하지 않고도 전기를 얻을 수 있어 친환경적이지요. 태양 에너지를 전기 에너지로 바꾸는 장치를 ‘태양 전지’라고 하는데요. 햇빛이 비추게 되면 태양 전지 안의 전자가 움직이면서 전류가 형성되는 원리로 전기를 사용할 수 있지요. 보통은 건물 옥상에 커다랗고 넓적한 모양의 태양 전지 장치를 건설해 에너지를 얻어요.

최근 이런 거대한 태양 전지 외에도 실생활에서 쉽게 에너지를 얻을 수 있는 작은 태양 전지가 개발되고 있어요. 울산과학기술원(UNIST·유니스트) 에너지화학공학과 서관용 교수 연구진이 개발한 태양 전지는 스마트폰 화면에 올려두고 자연광을 비추면 에너지를 얻을 수 있는 전지예요. 이 태양 전지의 또 다른 특징은 유리처럼 몸체가 투명하다는 거예요. 연구진은 “스마트폰을 비롯해 작은 모바일 기기 화면에서도 태양 전지를 사용할 수 있음을 확인했다”고 밝혔지요. 앞으로 연구를 거듭하면 _______________________ "
    "options":  {
        "1":"태양전지를 활용할 수 없어요",
        "2":"모바일 기기에 발전이 있을 것이에요",
        "3":"이 태양 전지를 스마트폰뿐만 아니라 자동차의 유리에도 활용할 수 있을 것으로 보여요.",
        "4":"스마트폰이 투명해질 거에요"
        },
    "answer": 3,
    "reason": "답에 대한 이유"
}
"""

responses = []
level_mapping = {
    1: "미취학아동",
    2: "초등학교 1학년",
    3: "초등학교 2학년",
    4: "초등학교 3학년",
    5: "초등학교 4학년",
    6: "초등학교 5학년",
    7: "초등학교 6학년"
}

for i, row in news.iterrows():
    if (i+1)%50==0:
        print(i+1)
    education_level = level_mapping.get(row['Difficulty_Level'], "Unknown")
    response = get_completion(row['preprocessed_content'], education_level)
    
    if response:
        # print(response)
        responses.append(response)
    else:
        responses.append("Failed to get a response from the API.")
        
# print(responses)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850


In [6]:
responses_df = pd.DataFrame(news[['URL','preprocessed_content', 'Difficulty_Level']])
responses_df['response'] = responses

responses_df.to_csv("../data/processed/news/dt03_21.csv", index=False, encoding='utf-8-sig')

In [5]:
# import pandas as pd
# responses_df = pd.read_csv("../data/processed/news/result/news_level.csv", sep=',')
# responses_df['textID'] = ['n{:04d}'.format(i+1) for i in range(len(responses_df))]
# responses_df.to_csv("../data/processed/news/result/news_level.csv", index=False, encoding='utf-8-sig')

In [10]:
#불러오기
import pandas as pd
import json
from datetime import datetime

responses_df = pd.read_csv("../data/processed/news/dt03_21.csv", sep=',')
news = pd.read_csv("../data/processed/news/result/news_level.csv", sep=',')
content_to_textID = dict(zip(news['preprocessed_content'], news['textID']))

print(responses_df['response'].head())
transformed_data = []

def get_textID(content):
    return content_to_textID.get(content, None)

def transform_row(row):
    response_json = json.loads(row['response'])
    options = response_json.get('options', {})
    
    answer_key = str(response_json.get("answer", ""))  
    answer_value = ""
    for k, v in options.items():
        if str(k) == answer_key:
            answer_value = v  
            break
    
    now = datetime.now().isoformat()

    transformed = {
        "title": response_json.get("question", ""),
        "content": response_json.get("text", ""),
        "answer" : answer_value,
        "solution": response_json.get("reason", ""),
        "questionTypeCode": "QT01",
        "problemLevelCode": f"PL0{row['Difficulty_Level']}",
        "problemTypeCode": "PT03",
        "problemTypeDetailCode": "PT0321",
        "metadata": {
            "options": [v for v in options.values()] 
        },
        "createdDate": now,
        "modifiedDate": now
    }
    
    return transformed


for i, row in responses_df.iterrows():
    try:
        json.loads(row['response'])  # Try parsing the original JSON
        transformed_data.append(transform_row(row))
    except json.JSONDecodeError as e:
        print(f"Error in row {i}: {e}")
        print(f"Problematic JSON: {row['response']}")

dt03_21_df = pd.DataFrame(transformed_data)

0    {\n    "question": "빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하...
1    {\n    "question": "빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하...
2    {\n    "question": "빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하...
3    {\n    "question": "빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하...
4    {\n    "question": "빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하...
Name: response, dtype: object
Error in row 14: Expecting ',' delimiter: line 4 column 5 (char 289)
Problematic JSON: {
    "question": "빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요",
    "text": "타이타닉호에 대한 인양권을 가진 미국의 기업 ‘RMS 타이타닉’이 최근 타이타닉호를 탐사하며 찍은 사진을 공개했습니다. 이번 탐사에서 로마 신화에 등장하는 사냥의 여신 다이애나의 모습을 담은 청동 조각상이 발견됐습니다. 1912년 당시 이 조각상은 타이타닉호 1등석 안에 있었지만 지금은 바다 깊은 곳 모래 바닥에 박혀 있습니다. 앞으로 이 조각상을 _______________________ "
    "options":  {
        "1":"다시 바다 깊은 곳에 숨길 것입니다.",
        "2":"타이타닉호와 함께 영원히 가라앉을 것입니다.",
        "3":"인양하여 사람들에게 보여줄 수 있을 것입니다.",
        "4":"신화 속 여신 다이애나와 관련된 이야기를 잃어버릴 것입니다."
        },
    "answer": 3,
    "reason": "타이타닉호의 인양권을 가진 회사가 탐사하며 조각상을 발견했으므로, 앞으로 조각상을 인양하여 사람들에게 보여줄 가능성이 높습니다."
}
Error in row 24: Expec

In [11]:
print(dt03_21_df)
with open("../data/processed/problem_data/pt0321.json", 'w', encoding='utf-8') as f:
    json.dump(transformed_data, f, indent=4, ensure_ascii=False)
dt03_21_df.to_csv("../data/processed/problem_data/pt0321.csv", index=False, encoding='utf-8-sig')

                            title  \
0    빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
1    빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
2    빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
3    빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
4    빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
..                            ...   
849  빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
850  빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
851  빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
852  빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
853  빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   

                                               content  \
0    푸른 호수 위에 노란 왕관을 쓴 거대한 랍스터가 둥실둥실 떠있어요. 커다란 집게발이...   
1    추석이 코앞으로 다가왔어요. 추석 때는 맛있는 전과 같은 요리를 준비하며 즐거운 연...   
2    활활 타오르는 태양이 내뿜는 빛은 지구로 쏟아져요. 이런 태양 빛을 모아 전기를 만...   
3    지난달 29일부터 열린 2024 파리 패럴림픽이 12일간의 여정을 마감하고, 9일 ...   
4    프랑스 파리의 에펠탑에는 2024 파리 올림픽을 기념하는 오륜 조형물이 걸려있어요....   
..                                                 ...   
849  ‘BNK부산은행 2024 부산세계탁구선수권대회’가 성황리에 막을 내렸습니다. 이번 ...   
850  머지않아 '배우'라는 직업이 인공지능(AI)으로 대체될지도 모르겠어요. AI로 영상...   
851  가습기 살균제 사건으로 피해자와 그의 가족들이 입은

In [1]:
import pandas as pd
import json

news = pd.read_csv("../data/processed/news/result/news_level_0321.csv", sep=',', encoding='cp949')

dt0321 = pd.read_csv("../data/processed/problem_data/pt0321.csv", sep=',')

dt0321['contentID'] = news['URL']
dt0321['classification'] = news['classification']
dt0321['word_count'] = news['word_count']
dt0321['word_avg'] = news['word_avg']
dt0321['word_hard'] = news['word_hard']
dt0321['length'] = news['length']
dt0321['Difficulty_Level'] = news['Difficulty_Level']

print(dt0321)

                            title  \
0    빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
1    빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
2    빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
3    빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
4    빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
..                            ...   
849  빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
850  빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
851  빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
852  빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   
853  빈칸에 들어갈 내용으로 가장 알맞는 것을 추론하세요   

                                               content  \
0    푸른 호수 위에 노란 왕관을 쓴 거대한 랍스터가 둥실둥실 떠있어요. 커다란 집게발이...   
1    추석이 코앞으로 다가왔어요. 추석 때는 맛있는 전과 같은 요리를 준비하며 즐거운 연...   
2    활활 타오르는 태양이 내뿜는 빛은 지구로 쏟아져요. 이런 태양 빛을 모아 전기를 만...   
3    지난달 29일부터 열린 2024 파리 패럴림픽이 12일간의 여정을 마감하고, 9일 ...   
4    프랑스 파리의 에펠탑에는 2024 파리 올림픽을 기념하는 오륜 조형물이 걸려있어요....   
..                                                 ...   
849  ‘BNK부산은행 2024 부산세계탁구선수권대회’가 성황리에 막을 내렸습니다. 이번 ...   
850  머지않아 '배우'라는 직업이 인공지능(AI)으로 대체될지도 모르겠어요. AI로 영상...   
851  가습기 살균제 사건으로 피해자와 그의 가족들이 입은

In [2]:
from datetime import datetime
def transform_row(row):
    now = datetime.now().isoformat()

    transformed = {
        "title": row['title'],
        "content": row['content'],
        "answer" : row['answer'],
        "solution": row['solution'],
        "questionTypeCode": row['questionTypeCode'],
        "problemLevelCode": row['problemLevelCode'],
        "problemTypeCode": row['problemTypeCode'],
        "problemTypeDetailCode": row['problemTypeDetailCode'],
        "metadata": row['metadata'],
        "createdDate": now,
        "modifiedDate": now,
        "contentID": row['contentID'],
        "classification": row['classification'],
        "wordCount" : row['word_count'],
        "wordAvg" : row['word_avg'],
        "wordHard" : row['word_hard'],
        "length" : row['length'],
        "Difficulty_Level" : row['Difficulty_Level']
    }
    
    return transformed

In [3]:
transformed_data = []
for i, row in dt0321.iterrows():
    try:
        transformed_data.append(transform_row(row))
    except json.JSONDecodeError as e:
        print(f"Error in row {i}: {e}")
dt0321_df = pd.DataFrame(transformed_data)

In [7]:
dt0321.to_csv("../data/processed/problem_data/pt0321.csv", index=False, encoding='utf-8-sig')